In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler

from joblib import Parallel, delayed

#Custome graph format style sheet
plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = True
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.eps'
if not 'location' in locals():
    #save location.  First one is for running on home PC, second for running on the work laptop.  May need to make a global change
    location = 'E:\\Documents\\Dan\\Code\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'
    #location = 'C:\\Users\\dhendrickson\\Documents\\github\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'



In [2]:
# Task Specific includes:

#import scipy.special as sp
import math
import matplotlib.animation as animation
import time
from numpy import inf
# Choose which EFIT_Class to use:
#import EFIT_Class as EFIT
#import EFIT_Class_StressLargerVelocity as EFIT
#import EFIT_Class_OrignalEqualGrid as EFIT
#import EFIT_Class_Parallel_EqualGrid as EFIT
#import EFIT_Class_VelocityLargerStress as EFIT
#import EFIT_Class_OrignalEqualGrid_axisFlipper as EFIT
#import EFIT_Class_EqualGrid1eq as EFIT
#import EFIT_Class_EqualGrid9eq as EFIT
import EFIT_Class_EqualGrid9eq9grid as EFIT
#import visvis as vv

In [3]:
# set Constants:
PoissonRatio = 0.3
YoungModulus = 20 * (10**9)
mu = 80 * (10**9)         #First Lame Parameter
lmbda = 2 * mu * PoissonRatio / (1 - 2 * PoissonRatio)     #second Lame Parameter
rho = 7800       #density kg/m^3

#Calculate speed of longitudinal and transverse waves
cl = np.sqrt((lmbda + 2* mu)/rho)
ct = np.sqrt(mu/rho)

#Choose ferquency to be used for excitment
frequency = 40000

#calculate wave length
omegal = cl / frequency
omegat = ct / frequency

#check max step size
dtmax = 1/ (max(cl,ct) * np.sqrt(3/(min(omegal,omegat)/10)))


# about 1foot (0.3m) of just the web of 175lbs rail 
# BeamLength = 0.3
# BeamHeight = 0.0762
# BeamWidth = 0.0381
BeamLength = 0.04
BeamHeight = 0.04
BeamWidth = 0.04

#Run for 6 Cycles:
runtime = 5.0 / frequency 

#Set time step and grid step to be 10 steps per frequency and ten steps per wavelength respectively
ts = (1 / frequency) / 20  #  5e-08  #time step
gs = min(omegal, omegat) / 50    #grid step

Tsteps = int(math.ceil(runtime / ts)) + 1       #total Time STeps

gl = int(math.ceil(BeamLength / gs))         #number of grid points
gh = int(math.ceil(BeamHeight / gs)) 
gw = int(math.ceil(BeamWidth / gs)) 

print(dtmax,ts)

8.622367091851726e-06 1.25e-06


In [4]:
def makeAnimation(CenterZResults, title='Z'):
    y = np.linspace(0, BeamHeight, np.shape(CenterZResults[0][0])[1])
    x = np.linspace(0, BeamLength, np.shape(CenterZResults[0][0])[0])
    x,y = np.meshgrid(x,y)

    fig = plt.figure(figsize=(6.0,BeamHeight/BeamLength*6.0), dpi=100)
    ax = plt.axes(xlim=(0, BeamLength), ylim=(0, BeamHeight))  
    plt.ylabel(r'height')
    plt.xlabel(r'length')

    # animation function
    def animate(i): 
        z = np.matrix(CenterZResults[i][0][:,:]).T
        plt.title(str(i) + ' : ' + "{:.3e}".format(CenterZResults[i][1]))
        cont = plt.contourf(x, y, z, levels=5, cmap='gray') #,vmin=-100, vmax=100)
        #time.sleep(1)
        return cont  

    anim = animation.FuncAnimation(fig, animate, frames=np.shape(CenterZResults)[0])

    anim.save('animation'+title+'.gif')

In [16]:
# Inputs for forcing Function
Power = 10          #not sure on unit yet, just something for now
EmitterSize = 0.01  # meters, so 1 CM
Dimmension = 2      # 2 is in the z axis 
Direction = 1       # 1 is on top going down
CornerCut = 3       # how much of the corner is taken off of the square emitter
StartStep = 2
EndStep = Tsteps

In [6]:
controls = []
for i in range(32):
    lba=[]
    Binary = str(int('{0:08b}'.format(i))).zfill(5)
    
    lba = [int(Binary[0]),int(Binary[1]),int(Binary[2]),int(Binary[3]),int(Binary[4])]
    
    #[sx,sy,sz,vx,vy,vz] = lba #[1,1,1,1,1,1]
    [c1,c2,c3,c4,c5] =np.multiply(lba,[-2,-2,-2,-2,-2])
    c1+=1
    c2+=1
    c3+=1
    c4+=1
    c5+=1
    
    controls.append([i, c1, c2, c3, c4, c5])

print(np.matrix(controls))

[[ 0  1  1  1  1  1]
 [ 1  1  1  1  1 -1]
 [ 2  1  1  1 -1  1]
 [ 3  1  1  1 -1 -1]
 [ 4  1  1 -1  1  1]
 [ 5  1  1 -1  1 -1]
 [ 6  1  1 -1 -1  1]
 [ 7  1  1 -1 -1 -1]
 [ 8  1 -1  1  1  1]
 [ 9  1 -1  1  1 -1]
 [10  1 -1  1 -1  1]
 [11  1 -1  1 -1 -1]
 [12  1 -1 -1  1  1]
 [13  1 -1 -1  1 -1]
 [14  1 -1 -1 -1  1]
 [15  1 -1 -1 -1 -1]
 [16 -1  1  1  1  1]
 [17 -1  1  1  1 -1]
 [18 -1  1  1 -1  1]
 [19 -1  1  1 -1 -1]
 [20 -1  1 -1  1  1]
 [21 -1  1 -1  1 -1]
 [22 -1  1 -1 -1  1]
 [23 -1  1 -1 -1 -1]
 [24 -1 -1  1  1  1]
 [25 -1 -1  1  1 -1]
 [26 -1 -1  1 -1  1]
 [27 -1 -1  1 -1 -1]
 [28 -1 -1 -1  1  1]
 [29 -1 -1 -1  1 -1]
 [30 -1 -1 -1 -1  1]
 [31 -1 -1 -1 -1 -1]]


In [17]:
#Run main function for time:
Animating = False

def loooop(c1=1,c2=1,c3=1,c4=1,c5=1,c6=1): 
    
    
    #Initialize EFIT Model
    Rail = EFIT.EFIT(gl, gw, gh, ts, gs)#, c1, c2) #, c3, c4, c5)

    #Set Material Properties consitant througout
    #Rail.Gp[0,:,:,:] = rho  #constant Density
    #Rail.Gp[1,:,:,:] = lmbda #Constant first Lamee parameter 
    #Rail.Gp[2,:,:,:] = mu  #constant second Lamee parameter
    
    Rail.rho = rho
    Rail.lmbda = lmbda
    Rail.me = mu

    #initial conditions are rest
    Rail.GvX[:,:,:]=0
    Rail.GvY[:,:,:]=0
    Rail.GvZ[:,:,:]=0
    Rail.GsXX[:,:,:]=0
    Rail.GsYY[:,:,:]=0
    Rail.GsZZ[:,:,:]=0
    Rail.GsXY[:,:,:]=0
    Rail.GsYZ[:,:,:]=0
    Rail.GsXZ[:,:,:]=0

    Rail.axisX = c1
    Rail.axisY = c2
    Rail.axisZ = c3
    Rail.StPl = c4
    Rail.VePl = c5

    CenterXResults = []
    CenterYResults = []
    CenterZResults = []

    t=0
    if Animating:
        CenterXResults.append((np.matrix(Rail.VelocityCut(0,2)),t))
        CenterYResults.append((np.matrix(Rail.VelocityCut(1,2)),t))
        CenterZResults.append((np.matrix(Rail.VelocityCut(2,2)),t))
    else:
        CenterXResults.append(np.matrix(Rail.VelocityCut(0,2)))
        CenterYResults.append(np.matrix(Rail.VelocityCut(1,2)))
        CenterZResults.append(np.matrix(Rail.VelocityCut(2,2)))



    for i in range(Tsteps - 1):
        t = (i) * ts
    
        #update Velocity:
        Rail.StepVelocities()
        
        if i >= StartStep and i <= EndStep: 
            #Rail.ForcingFunctionImpulse(Power,EmitterSize,Dimmension,Direction, CornerCut)
            Rail.ForcingFunctionWave(t, frequency, 1, 0.005, 2, 2)
        #else:
        #    Rail.ForcingFunctionWave(t, frequency, 0)

        #Update Stresses at next half step:
        Rail.StepStresses()

        
        #print(str(i+1) + ' of ' + str(Tsteps-1) +' time steps. time is: '+ "{:.3e}".format(t)) #str(t))

        #  Save off each time step the currrent state for anmication later
        if Animating:
            CenterXResults.append((np.matrix(Rail.VelocityCut(0,2)),t))
            CenterYResults.append((np.matrix(Rail.VelocityCut(1,2)),t))
            CenterZResults.append((np.matrix(Rail.VelocityCut(2,2)),t))
        else:
            CenterXResults.append(np.matrix(Rail.VelocityCut(0,2)))
            CenterYResults.append(np.matrix(Rail.VelocityCut(1,2)))
            CenterZResults.append(np.matrix(Rail.VelocityCut(2,2)))
        # AllVelocities.append( Rail.VelocitySave())
        # VelocitiesX.append(Rail.VelocitySave(0))
        # VelocitiesY.append(Rail.VelocitySave(1))
        # VelocitiesZ.append(Rail.VelocitySave(2))
        
        # Store results mid process for latter animating
        #if i % 10 == 9:
        #    print(str(i+1) + ' of ' + str(Tsteps-1) +' time steps. time is: '+ "{:.3e}".format(t)+' on loop '+str(looooooping)) #str(t))
        
        # Other data save out options at different time steps

    if Animating:
        makeAnimation(CenterXResults, ' try 4X '+str(c6)) #+str(vx)+str(vy)+str(vz))
        #makeAnimation(CenterYResults, ' try 4Y '+str(c6)) #+str(vx)+str(vy)+str(vz))
        #makeAnimation(CenterZResults, ' try 4Z '+str(c6)) #+str(vx)+str(vy)+str(vz))

    return CenterXResults,CenterYResults,CenterZResults


In [18]:
#Results = loooop(1,1,1)
Results = Parallel(n_jobs=12)(delayed(loooop)(c[1],c[2],c[3],c[4],c[5], c[0]) for c in controls)
#for i in range(8):
#    Results = loooop(i)
#    for j in range(20):
#        print(np.sum(Results[0][j*8][0]),np.sum(Results[1][j*8][0]),np.sum(Results[2][j*8][0]))

In [9]:
np.shape(Results)

(32, 3, 101, 25, 25)

In [19]:
if not Animating:
    Summary = []
    for set in Results:
        maxX = np.max(np.max(set[0]))
        maxY = np.max(np.max(set[1]))
        maxZ = np.max(np.max(set[2]))
        Summary.append([maxX,maxY,maxZ])
    print(Summary)

[[5.6593955548439205e+214, 2.236821112669509e+215, 7.232709416814157e+214], [4.756797786777188e+215, 1.826256964366133e+216, 6.0753278085088175e+215], [4.756797786777188e+215, 1.826256964366133e+216, 6.0753278085088175e+215], [5.6593955548439205e+214, 2.236821112669509e+215, 7.232709416814157e+214], [2.124157851348468e+178, 2.124169776367744e+178, 9.389528055375426e+173], [3.4078566854297957e+180, 3.4078566854297957e+180, 1.3283631273579826e+175], [3.4078566854297957e+180, 3.4078566854297957e+180, 1.3283631273579826e+175], [2.124157851348468e+178, 2.124169776367744e+178, 9.389528055375426e+173], [1.0992594943569131e+191, 1.0802783532567137e+193, 1.5610038594232177e+192], [2.1118512628152235e+189, 2.243115436992708e+191, 3.297846398521344e+190], [2.1118512628152235e+189, 2.243115436992708e+191, 3.297846398521344e+190], [1.0992594943569131e+191, 1.0802783532567137e+193, 1.5610038594232177e+192], [1.5296347838938761e+178, 1.2811978893408153e+178, 4.258541598674044e+177], [3.05294458045806

def ThreeDimmAnimate(FourDVector):
    # create volumes, loading them into opengl memory, and insert into container.
    f = vv.clf()
    a = vv.gca()
    m = vv.MotionDataContainer(a)

    for vol in FourDVector:
        t = vv.volshow(vol)
        t.parent = m
        t.colormap = vv.CM_HOT
        # Remove comments to use iso-surface rendering
        #t.renderStyle = 'iso'
        #t.isoThreshold = 0.2

    # set some settings
    a.daspect = 1,1,-1
    a.xLabel = 'x'
    a.yLabel = 'y'
    a.zLabel = 'z'

    # Enter main loop
    app = vv.use()
    app.Run()

In [12]:
#ThreeDimmAnimate(AllVelocities)

In [13]:
#ThreeDimmAnimate(VelocitiesX)

In [14]:
#ThreeDimmAnimate(VelocitiesY)

In [15]:
#ThreeDimmAnimate(VelocitiesZ)